In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install yfinance

In [3]:
# --- 附加题：Bitcoin "纯净版" 数据收集脚本 ---
# 目标：收集 BTC 数据，不引入任何外部数据源，确保无因对齐产生的填充值

import yfinance as yf
import pandas as pd
import numpy as np
import datetime

# 1. 设置参数
TICKER = "BTC-USD"
# 获取足够长的数据以包含训练和测试
START_DATE = "2018-01-01" 
END_DATE = datetime.datetime.now().strftime('%Y-%m-%d')

print(f"--- 正在下载 {TICKER} 历史数据 ---")

# 2. 下载数据 (Yahoo Finance)
# auto_adjust=True 确保我们拿到的是复权后的真实价格
raw_data = yf.download(TICKER, start=START_DATE, end=END_DATE, auto_adjust=True)

# 检查数据是否为空
if len(raw_data) == 0:
    raise ValueError("下载失败，请检查网络或 Ticker 名称")

# 3. 数据清洗与重组
df = raw_data.copy()

# 重命名为标准 OHLCV 格式
# 注意：yfinance 新版有时候返回多级索引，这里做一下处理
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)
    
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

# 4. 构建符合项目要求的特征列 (P, V)
# 我们直接使用原始价格，不进行任何填充
df['P1_Close'] = df['Close']
df['P2_Open'] = df['Open']
df['P3_High'] = df['High']
df['P4_Low'] = df['Low']
df['V1_Volume'] = df['Volume']

# 5. 构建目标变量 (Target)
# PDF  允许预测 "returns" (原始收益) 而不仅是超额收益。
# 为了保证数据纯净，我们不引入外部的 "无风险利率"，直接预测绝对收益。
# 收益率 = (今天收盘 - 昨天收盘) / 昨天收盘
df['daily_return'] = df['Close'].pct_change()

# 目标：预测 *下一天* 的收益 (Forward Return)
df['market_forward_excess_returns'] = df['daily_return'].shift(-1)

# 为了兼容您的主任务代码，我们需要一个 risk_free_rate 列。
# 为了不编造数据，我们将这列设为 0 (意味着我们预测的是绝对收益，而非超额收益)
df['risk_free_rate'] = 0.0

# 6. 生成 date_id 和 Is_Scored
# 移除包含 NaN 的行 (通常是第一行和最后一行)
df_clean = df.dropna().copy()
df_clean['date_id'] = range(len(df_clean))

# 7. 数据集情报输出
print("\n" + "="*40)
print(f"📊 {TICKER} 数据集情报")
print("="*40)
print(f"1. 形状: {df_clean.shape}")
print(f"2. 时间覆盖: {df_clean.index.min().date()} 到 {df_clean.index.max().date()}")
print(f"3. 缺失值检查: {df_clean.isnull().sum().sum()} (必须为 0)")
print("4. 数据预览:")
print(df_clean[['date_id', 'P1_Close', 'market_forward_excess_returns']].tail())

# 8. 保存
filename = "bonus_btc_pure.csv"
df_clean.to_csv(filename)
print(f"\n✅ 数据已保存为 '{filename}'")
print("提示：在您的主代码中，将 train_path 指向此文件即可。")

--- 正在下载 BTC-USD 历史数据 ---


[*********************100%***********************]  1 of 1 completed


📊 BTC-USD 数据集情报
1. 形状: (2885, 14)
2. 时间覆盖: 2018-01-02 到 2025-11-25
3. 缺失值检查: 0 (必须为 0)
4. 数据预览:
Price       date_id      P1_Close  market_forward_excess_returns
Date                                                            
2025-11-21     2880  85090.687500                      -0.005198
2025-11-22     2881  84648.359375                       0.025478
2025-11-23     2882  86805.007812                       0.016883
2025-11-24     2883  88270.562500                      -0.010521
2025-11-25     2884  87341.890625                       0.036368

✅ 数据已保存为 'bonus_btc_pure.csv'
提示：在您的主代码中，将 train_path 指向此文件即可。
